In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from proj2_helpers import *
# from sklearn import linear_model
# from sklearn.preprocessing import PolynomialFeatures
from helpers_CNN import *

import torch.utils.data as utils
import time
import torch
from torch import nn, optim
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import torch.nn.functional as F
print(torch.__version__)
print(device)

1.3.1
cpu


In [4]:
# Loaded a set of images
num_traset = 100 #Trainingset numbers
root_dir = "training_1"

image_dir = root_dir + "/images/"
# files = os.listdir(image_dir)
files = [f for f in os.listdir(image_dir) if not f.startswith('.')]
files.sort()
n = min(num_traset,len(files)) 
print("Loading " + str(n) + " images")
imgs = np.asarray([load_image(image_dir + files[i]) for i in range(n)])
print(files[0])
gt_dir = root_dir + "/groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = np.asarray([load_image(gt_dir + files[i]) for i in range(n)])
print(files[0])

Loading 100 images
satImage_001.png
Loading 100 images
satImage_001.png


In [26]:
# print(files)

In [6]:
window_size = 72
patch_size = 16
padding_size = (window_size - patch_size) // 2
imag_w = imgs[0].shape[0]
imag_h = imgs[0].shape[1]
num_ch = imgs[0].shape[2]

In [7]:
imgs[0].shape

(400, 400, 3)

In [8]:
# Padding training images
imgs_pad = np.empty((len(imgs),imag_w+2*padding_size, imag_h+2*padding_size, num_ch))
#gt_imgs_pad = np.empty((len(imgs),imag_w+2*padding_size, imag_h+2*padding_size))
for i in range(len(imgs)):
    imgs_pad[i] = pad_image(imgs[i], padding_size)
    #gt_imgs_pad[i] = pad_image(gt_imgs[i], padding_size) #Patches don't need padding
    

In [9]:
stride = 16
img_windows = np.asarray([img_crop_stride(imgs_pad[i], window_size, window_size, stride) for i in range(num_traset)])
gt_patches = np.asarray([img_crop_stride(gt_imgs[i], patch_size, patch_size, stride) for i in range(num_traset)])

In [10]:
img_windows.shape

(100, 625, 72, 72, 3)

In [11]:
# Linearize list of patches
img_windows_ln = [img_windows[i][j] for i in range(len(img_windows)) for j in range(len(img_windows[i]))]
gt_patches_ln =  [gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))]
#img_tra_np = np.array(img_windows_ln)
#gt_tra_np = np.array(gt_patches_ln)

In [12]:
# Extract labels for each image patch
labels_tra_list = [value_to_class(np.mean(gt_patches_ln[i])) for i in range(len(gt_patches_ln))]
#labels_tra_np = np.asarray([value_to_class(np.mean(gt_patches_ln[i])) for i in range(len(gt_patches_ln))])

In [13]:
len(labels_tra_list)

62500

In [14]:
# Creat dataset
img_tra = torch.stack([torch.Tensor(imgs).float() for imgs in img_windows_ln]) # transform to torch tensors
labels_tra = torch.Tensor(labels_tra_list).long()#Labels should be long type

In [15]:
print(labels_tra.dtype,img_tra.dtype)

torch.int64 torch.float32


In [16]:
img_tra_ncwh = img_tra.permute(0,3,1,2)

In [17]:
tra_dataset = utils.TensorDataset(img_tra_ncwh,labels_tra) # create your datset

In [18]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 6, 5), # in_channels, out_channels, kernel_size
            #nn.Sigmoid(),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            #nn.Sigmoid(),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*15*15, 120),
            #nn.Sigmoid(),
            nn.ReLU(),
            nn.Linear(120, 84),
            #nn.Sigmoid(),
            nn.ReLU(),
            nn.Linear(84, 2)
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

In [19]:
# Training
def train_lenet(net, train_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            train_l_new, train_acc_new = 0.0, 0.0
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_new = l.cpu().item()
            train_l_sum += train_l_new
            train_acc_new = (y_hat.argmax(dim=1) == y).sum().cpu().item()/y.shape[0]
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
            print('loss new %.4f, train acc new %.3f' %(train_l_new, train_acc_new) )
        print('epoch %d, loss %.4f, train acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n,  time.time() - start))

In [20]:
# Load data
num_workers=1
batch_size = 256
train_iter = utils.DataLoader(tra_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [21]:
net = LeNet().float()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=3600, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=2, bias=True)
  )
)


In [22]:
lr, num_epochs = 0.001, 10
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
#optimizer = torch.optim.SGD(net.parameters(), lr=lr)
train_lenet(net, train_iter, batch_size, optimizer, device, num_epochs)

training on  cpu
loss new 0.6911, train acc new 0.707
loss new 0.6148, train acc new 0.777
loss new 0.6010, train acc new 0.711
loss new 0.5688, train acc new 0.750
loss new 0.5200, train acc new 0.785
loss new 0.5574, train acc new 0.754
loss new 0.5837, train acc new 0.738
loss new 0.5427, train acc new 0.766
loss new 0.5581, train acc new 0.758
loss new 0.5642, train acc new 0.746
loss new 0.6002, train acc new 0.707
loss new 0.5123, train acc new 0.801
loss new 0.5791, train acc new 0.727
loss new 0.5249, train acc new 0.773
loss new 0.5100, train acc new 0.785
loss new 0.5712, train acc new 0.750
loss new 0.5492, train acc new 0.758
loss new 0.5173, train acc new 0.777
loss new 0.5309, train acc new 0.773
loss new 0.5806, train acc new 0.719
loss new 0.5447, train acc new 0.754
loss new 0.5798, train acc new 0.723
loss new 0.5562, train acc new 0.738
loss new 0.5605, train acc new 0.730
loss new 0.5125, train acc new 0.773
loss new 0.6481, train acc new 0.680
loss new 0.5672, trai

loss new 0.4375, train acc new 0.785
loss new 0.4554, train acc new 0.812
loss new 0.4631, train acc new 0.773
loss new 0.5087, train acc new 0.754
loss new 0.4636, train acc new 0.754
loss new 0.4715, train acc new 0.797
loss new 0.4240, train acc new 0.777
loss new 0.4253, train acc new 0.793
loss new 0.4895, train acc new 0.742
loss new 0.4558, train acc new 0.789
loss new 0.4300, train acc new 0.809
loss new 0.4431, train acc new 0.781
loss new 0.4475, train acc new 0.762
loss new 0.4558, train acc new 0.781
loss new 0.4163, train acc new 0.809
loss new 0.4981, train acc new 0.742
loss new 0.4769, train acc new 0.758
loss new 0.4869, train acc new 0.766
loss new 0.4627, train acc new 0.773
loss new 0.4443, train acc new 0.781
loss new 0.4527, train acc new 0.797
loss new 0.4700, train acc new 0.777
loss new 0.4984, train acc new 0.762
loss new 0.4366, train acc new 0.833
epoch 1, loss 0.4959, train acc 0.756, time 82.1 sec
loss new 0.4581, train acc new 0.793
loss new 0.3950, train

loss new 0.3841, train acc new 0.820
loss new 0.3196, train acc new 0.848
loss new 0.3394, train acc new 0.816
loss new 0.3995, train acc new 0.797
loss new 0.4085, train acc new 0.832
loss new 0.4047, train acc new 0.809
loss new 0.4118, train acc new 0.781
loss new 0.3787, train acc new 0.797
loss new 0.3952, train acc new 0.805
loss new 0.3686, train acc new 0.844
loss new 0.3889, train acc new 0.809
loss new 0.4088, train acc new 0.809
loss new 0.3456, train acc new 0.863
loss new 0.4596, train acc new 0.785
loss new 0.3961, train acc new 0.816
loss new 0.3882, train acc new 0.836
loss new 0.4011, train acc new 0.805
loss new 0.4341, train acc new 0.805
loss new 0.3950, train acc new 0.777
loss new 0.3643, train acc new 0.859
loss new 0.3783, train acc new 0.820
loss new 0.2980, train acc new 0.871
loss new 0.3819, train acc new 0.816
loss new 0.4095, train acc new 0.824
loss new 0.4563, train acc new 0.773
loss new 0.3328, train acc new 0.832
loss new 0.3967, train acc new 0.801
l

loss new 0.3823, train acc new 0.816
loss new 0.3205, train acc new 0.855
loss new 0.3199, train acc new 0.867
loss new 0.2822, train acc new 0.891
loss new 0.2941, train acc new 0.879
loss new 0.3079, train acc new 0.863
loss new 0.3099, train acc new 0.844
loss new 0.3298, train acc new 0.867
loss new 0.3127, train acc new 0.859
loss new 0.3600, train acc new 0.836
loss new 0.3446, train acc new 0.855
loss new 0.2987, train acc new 0.871
loss new 0.3887, train acc new 0.812
loss new 0.3235, train acc new 0.863
loss new 0.3362, train acc new 0.867
loss new 0.3399, train acc new 0.852
loss new 0.3147, train acc new 0.824
loss new 0.3786, train acc new 0.855
loss new 0.3156, train acc new 0.844
loss new 0.2732, train acc new 0.895
loss new 0.3211, train acc new 0.875
loss new 0.3073, train acc new 0.832
loss new 0.3477, train acc new 0.844
loss new 0.2760, train acc new 0.867
loss new 0.2776, train acc new 0.898
loss new 0.3478, train acc new 0.855
loss new 0.3521, train acc new 0.836
l

loss new 0.2835, train acc new 0.887
loss new 0.2805, train acc new 0.871
loss new 0.2577, train acc new 0.891
loss new 0.2977, train acc new 0.867
loss new 0.2773, train acc new 0.871
loss new 0.2643, train acc new 0.895
loss new 0.2637, train acc new 0.879
loss new 0.2328, train acc new 0.902
loss new 0.3287, train acc new 0.848
loss new 0.3076, train acc new 0.867
loss new 0.2481, train acc new 0.910
loss new 0.2760, train acc new 0.891
loss new 0.3660, train acc new 0.848
loss new 0.3192, train acc new 0.844
loss new 0.2797, train acc new 0.852
loss new 0.2256, train acc new 0.898
loss new 0.3289, train acc new 0.887
loss new 0.3055, train acc new 0.863
loss new 0.2524, train acc new 0.898
loss new 0.2678, train acc new 0.875
loss new 0.2700, train acc new 0.883
loss new 0.3133, train acc new 0.867
loss new 0.3269, train acc new 0.844
loss new 0.2880, train acc new 0.891
loss new 0.2885, train acc new 0.875
loss new 0.3010, train acc new 0.859
loss new 0.2296, train acc new 0.922
l

...

loss new 0.2072, train acc new 0.930
loss new 0.1847, train acc new 0.926
loss new 0.1650, train acc new 0.930
loss new 0.1642, train acc new 0.949
loss new 0.1846, train acc new 0.906
loss new 0.2374, train acc new 0.910
loss new 0.1683, train acc new 0.934
loss new 0.1978, train acc new 0.918
loss new 0.1675, train acc new 0.938
loss new 0.1736, train acc new 0.941
loss new 0.1455, train acc new 0.949
loss new 0.2508, train acc new 0.898
loss new 0.1876, train acc new 0.934
loss new 0.2655, train acc new 0.898
loss new 0.1809, train acc new 0.926
loss new 0.1913, train acc new 0.910
loss new 0.1926, train acc new 0.922
loss new 0.2136, train acc new 0.898
loss new 0.1873, train acc new 0.918
loss new 0.1866, train acc new 0.938
loss new 0.1843, train acc new 0.926
loss new 0.2271, train acc new 0.891
loss new 0.2007, train acc new 0.941
loss new 0.2295, train acc new 0.891
loss new 0.2070, train acc new 0.910
loss new 0.2043, train acc new 0.914
loss new 0.2045, train acc new 0.922
l

loss new 0.1639, train acc new 0.922
loss new 0.2123, train acc new 0.906
loss new 0.1420, train acc new 0.938
loss new 0.0863, train acc new 0.973
loss new 0.1144, train acc new 0.953
loss new 0.1513, train acc new 0.953
loss new 0.1029, train acc new 0.961
loss new 0.1542, train acc new 0.930
loss new 0.1706, train acc new 0.941
loss new 0.1503, train acc new 0.941
loss new 0.1821, train acc new 0.910
loss new 0.1319, train acc new 0.949
loss new 0.1563, train acc new 0.930
loss new 0.1171, train acc new 0.961
loss new 0.1046, train acc new 0.969
loss new 0.1374, train acc new 0.938
loss new 0.1703, train acc new 0.926
loss new 0.1897, train acc new 0.910
loss new 0.1390, train acc new 0.934
loss new 0.1848, train acc new 0.902
loss new 0.1619, train acc new 0.934
loss new 0.1873, train acc new 0.934
loss new 0.1466, train acc new 0.934
loss new 0.1798, train acc new 0.949
loss new 0.1532, train acc new 0.934
loss new 0.1681, train acc new 0.938
loss new 0.1524, train acc new 0.949
l

loss new 0.1226, train acc new 0.961
loss new 0.1074, train acc new 0.961
loss new 0.1208, train acc new 0.949
loss new 0.1202, train acc new 0.953
loss new 0.1761, train acc new 0.930
loss new 0.1722, train acc new 0.938
loss new 0.1638, train acc new 0.934
loss new 0.1516, train acc new 0.945
loss new 0.1349, train acc new 0.957
loss new 0.1210, train acc new 0.934
loss new 0.1558, train acc new 0.938
loss new 0.1572, train acc new 0.945
loss new 0.1346, train acc new 0.945
loss new 0.1502, train acc new 0.945
loss new 0.1395, train acc new 0.945
loss new 0.1412, train acc new 0.938
loss new 0.1205, train acc new 0.961
loss new 0.1061, train acc new 0.961
loss new 0.1156, train acc new 0.969
loss new 0.1064, train acc new 0.969
loss new 0.0856, train acc new 0.977
loss new 0.1321, train acc new 0.934
loss new 0.1567, train acc new 0.934
loss new 0.1471, train acc new 0.930
loss new 0.1310, train acc new 0.953
loss new 0.1878, train acc new 0.910
loss new 0.1007, train acc new 0.957
l

loss new 0.1859, train acc new 0.926
loss new 0.1964, train acc new 0.944
epoch 9, loss 0.1518, train acc 0.938, time 76.5 sec
loss new 0.1316, train acc new 0.938
loss new 0.1221, train acc new 0.965
loss new 0.1571, train acc new 0.949
loss new 0.1435, train acc new 0.945
loss new 0.2075, train acc new 0.922
loss new 0.1494, train acc new 0.938
loss new 0.1309, train acc new 0.949
loss new 0.1444, train acc new 0.941
loss new 0.0851, train acc new 0.969
loss new 0.1206, train acc new 0.953
loss new 0.1605, train acc new 0.945
loss new 0.1569, train acc new 0.922
loss new 0.2250, train acc new 0.898
loss new 0.1577, train acc new 0.934
loss new 0.1247, train acc new 0.941
loss new 0.1422, train acc new 0.938
loss new 0.1217, train acc new 0.953
loss new 0.1193, train acc new 0.961
loss new 0.1279, train acc new 0.945
loss new 0.1513, train acc new 0.934
loss new 0.1560, train acc new 0.945
loss new 0.1201, train acc new 0.930
loss new 0.1310, train acc new 0.949
loss new 0.1258, train

loss new 0.1992, train acc new 0.910
loss new 0.1198, train acc new 0.949
loss new 0.1553, train acc new 0.938
loss new 0.1339, train acc new 0.941
loss new 0.1836, train acc new 0.922
loss new 0.1422, train acc new 0.934
loss new 0.1569, train acc new 0.945
loss new 0.1913, train acc new 0.922
loss new 0.1100, train acc new 0.953
loss new 0.1733, train acc new 0.930
loss new 0.1542, train acc new 0.941
loss new 0.2393, train acc new 0.891
loss new 0.1698, train acc new 0.934
loss new 0.1249, train acc new 0.965
loss new 0.1613, train acc new 0.926
loss new 0.1163, train acc new 0.953
loss new 0.1368, train acc new 0.953
loss new 0.1731, train acc new 0.918
loss new 0.1933, train acc new 0.922
loss new 0.0882, train acc new 0.961
loss new 0.0908, train acc new 0.965
loss new 0.1299, train acc new 0.957
loss new 0.1164, train acc new 0.945
loss new 0.0997, train acc new 0.969
loss new 0.1286, train acc new 0.938
loss new 0.1580, train acc new 0.926
loss new 0.1052, train acc new 0.972
e

In [23]:
def get_prediction_lenet(net,img,window_size,padding_size,stride,IMG_PATCH_SIZE):  #For prediction, stride need to be 16
    imgs_pad = pad_image(img, padding_size)
    data = np.asarray(img_crop_stride(imgs_pad, window_size, window_size,stride))
    X_ts = torch.tensor(data).permute(0,3,1,2) #Change orders to ncwh
    Yscore_ts = net(X_ts)
    Y_ts = Yscore_ts.argmax(dim=1)
    Y_np = Y_ts.numpy()
    img_prediction = label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, Y_np)

    return img_prediction

In [24]:
window_size

72

In [25]:
# Get prediction
TEST_SIZE = 50
test_data_filename = 'Datasets/test_set_images/'
print("Running prediction on test set")
prediction_test_dir = "predictions_test/"
if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)
for i in range(1, TEST_SIZE + 1):
    test = "test_%d" % i
    image_filename = test_data_filename + test + '/' + test +  ".png"
    img = mpimg.imread(image_filename)
    pimg = get_prediction_lenet(net,img,window_size,padding_size,stride,patch_size)
    pimg8 = img_float_to_uint8(pimg)
    pimg8_L = Image.fromarray(pimg8, 'L')
    pimg8_L.save(prediction_test_dir + "prediction_" + str(i) + ".png")

Running prediction on test set


In [ ]:
imgs_pad = pad_image(imgs[1], padding_size)
data = np.asarray(img_crop_stride(imgs_pad, window_size, window_size,stride))
#X_ts = torch.tensor(data).permute(0,3,1,2) #Change orders to ncwh
#Yscore_ts = net(X_ts)
#Y_ts = Yscore_ts.argmax(dim=1)
#Y_np = Y_ts.numpy()
#img_prediction = label_to_img(imgs[1].shape[0], imgs[1].shape[1], patch_size, patch_size, Y_np)

In [ ]:
data.shape

In [ ]:
X1,Y1 = 

In [ ]:
test_data_filename = 'Datasets/test_set_images/'
test = "test_%d" % 1
image_filename_pic1 = test_data_filename + test + '/' + test +  ".png"
test_img = mpimg.imread(image_filename_pic1)

imgs_pad = pad_image(test_img, padding_size)
data = np.asarray(img_crop_stride(imgs_pad, window_size, window_size,stride))
X_ts = torch.tensor(data).permute(0,3,1,2) #Change orders to ncwh
Yscore_ts = net(X_ts)
Y_ts = Yscore_ts.argmax(dim=1)
Y_np = Y_ts.numpy()
img_prediction = label_to_img(img.shape[0], img.shape[1], patch_size, patch_size, Y_np)




#pimg = get_prediction_lenet(net,test_img,window_size,padding_size,stride,patch_size)
pimg8 = img_float_to_uint8(img_prediction)
pimg8_L = Image.fromarray(pimg8, 'L')

In [ ]:
img_prediction.shape

In [ ]:
data.shape

In [ ]:
pimg.view[1,-1]

In [ ]:
Y_np

In [ ]:
test = "test_%d" % 1
image_filenameXX = test_data_filename + test + '/' + test +  ".png"
img = mpimg.imread(image_filenameXX)
pimgXX = get_prediction_lenet(net,img,window_size,padding_size,stride,patch_size)
pimg8 = img_float_to_uint8(pimg)
#pimg8_L = Image.fromarray(pimg8, 'L')
#pimg8_L.save(prediction_test_dir + "prediction_" + str(i) + ".png")

In [ ]:
pimgXX

In [ ]:
img_windows_ln = np.empty((img_windows.shape[0]*img_windows.shape[1], window_size, window_size, 3))
gt_patches_ln = np.empty((gt_patches.shape[0]*gt_patches.shape[1], patch_size, patch_size))
for i in range(img_windows.shape[0]):
    for j in range(img_windows[i].shape[0]):
        img_windows_ln[i*img_windows.shape[0]+j] = img_windows[i][j]
        gt_patches_ln[i*gt_patches.shape[0]+j] = gt_patches[i][j]

In [ ]:
test0 = img_crop_stride(imgs_pad[0], 72, 72, 16)

In [ ]:
test0_ts = torch.tensor(test0).permute(0,3,1,2).float()

In [ ]:
test0_ts.numpy().shape

In [ ]:
test0_y_ts = net(test0_ts)

In [ ]:
test0_y_ts

In [ ]:
test_data_filename = 'Datasets/test_set_images/'

In [ ]:
test[1]

In [ ]:
img_patches = [img_crop_update(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop_gt(gt_imgs[i], patch_size, patch_size) for i in range(n)]

In [ ]:
# Extract patches from input images
patch_size = 72 # each patch is 72*72 pixels
stride = 8
for i in range(imgs_pad.shape[0]):
    imgs_patches = img_crop_update(imgs_pad[i], patch_size, patch_size,stride,0)
    gt_patches = img_crop_gt(gt_imgs_pad[i], patch_size, patch_size,stride)

In [ ]:
# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [ ]:
# Linearize list of patches
img_patches = np.asarray([imgs_patches[i][j] for i in range(len(imgs_patches)) for j in range(len(imgs_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [ ]:
img_patches.shape

In [ ]:
# Training
def train_cnn(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))
